In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm

In [25]:
df = pd.read_csv("intersection_animalia_inaturalist.csv")
df.head()

,Common name,Species,Full taxonomy,Description
0,hoatzin,opisthocomus hoazin,Animalia_Chordata_Aves_Opisthocomiformes_Opist...,The Hoatzin is a tropical bird native to South...
1,common musk turtle,sternotherus odoratus,Animalia_Chordata_Vertebrata_Reptilia_Testudin...,The Common musk turtle is a small turtle nativ...
2,harbor seal,phoca vitulina,Animalia_Chordata_Vertebrata_Mammalia_Carnivor...,The Harbor seal is relatively small in compari...
3,blue monkey,cercopithecus mitis,Animalia_Chordata_Vertebrata_Mammalia_Primates...,"Despite what the common name suggests, the Blu..."
4,carpet python,morelia spilota,Animalia_Chordata_Vertebrata_Reptilia_Squamata...,The Carpet python is a large snake named for b...


In [87]:
f = open('train.json',) 
train_annotations = json.load(f) 

In [19]:
train_annotations.keys()

dict_keys(['info', 'images', 'categories', 'annotations', 'licenses'])

In [27]:
intersect_species = set()
for species in df['Species']:
    intersect_species.add(species)

In [48]:
len(intersect_species)

673

In [94]:
intersect_json = json.loads('{}')
intersect_json['images'] = []

In [95]:
N = len(train_annotations['images'])
for i in tqdm(range(N)):
    image = train_annotations['images'][i]
    cid = train_annotations['annotations'][image['id']]['category_id']
    name = train_annotations['categories'][cid]['name']
    if name.lower() in intersect_species:
        intersect_json['images'].append(image.copy())

  0%|          | 0/2686843 [00:00<?, ?it/s]

In [96]:
intersect_json['categories'] = []
N = len(train_annotations['categories'])
for i in tqdm(range(N)):
    cat = train_annotations['categories'][i]
    name = cat['name']
    if name.lower() in intersect_species:
        intersect_json['categories'].append(cat.copy())

  0%|          | 0/10000 [00:00<?, ?it/s]

In [97]:
intersect_json['annotations'] = []
N = len(train_annotations['annotations'])
for i in tqdm(range(N)):
    ann = train_annotations['annotations'][i]
    cat_id = ann['category_id']
    name = train_annotations['categories'][cat_id]['name']
    if name.lower() in intersect_species:
        intersect_json['annotations'].append(ann.copy())

  0%|          | 0/2686843 [00:00<?, ?it/s]

In [98]:
# fix id numbers, maps below from 'old' to 'new' ids
cat_id_map = {}
image_id_map = {}

for new_id, cat in enumerate(intersect_json['categories']):
    old = cat['id']
    cat_id_map[old] = new_id
    cat['id'] = new_id
    
for new_id, im in enumerate(intersect_json['images']):
    old = im['id']
    image_id_map[old] = new_id
    im['id'] = new_id

for ann in intersect_json['annotations']:
    old = ann['id']
    ann['id'] = image_id_map[old]
    ann['image_id'] = image_id_map[old]
    ann['category_id'] = cat_id_map[ann['category_id']]

In [65]:
for a in intersect_json['annotations']:
    assert a['id'] == a['image_id']

In [99]:
intersect_json['images'][:3]

[{'id': 0,
  'width': 500,
  'height': 333,
  'file_name': 'train/04831_Animalia_Chordata_Mammalia_Rodentia_Sciuridae_Marmota_flaviventris/4653466f-f7b5-4e6c-985e-e059c1493b8f.jpg',
  'license': 0,
  'rights_holder': 'Michelle S. Koo',
  'date': '2010-07-06 22:17:00+00:00',
  'latitude': 38.86995,
  'longitude': -120.19326,
  'location_uncertainty': None},
 {'id': 1,
  'width': 500,
  'height': 375,
  'file_name': 'train/05015_Animalia_Chordata_Reptilia_Squamata_Iguanidae_Dipsosaurus_dorsalis/277c6a90-0432-4472-96b6-ed51a5cd21cf.jpg',
  'license': 1,
  'rights_holder': '105615097470186309865',
  'date': '2009-05-04 00:00:00+00:00',
  'latitude': 35.14218,
  'longitude': -116.10415,
  'location_uncertainty': None},
 {'id': 2,
  'width': 500,
  'height': 375,
  'file_name': 'train/05163_Animalia_Chordata_Reptilia_Testudines_Testudinidae_Gopherus_agassizii/49c31f70-59b9-45ed-bba7-83df3b728912.jpg',
  'license': 1,
  'rights_holder': 'biosam',
  'date': '2009-05-04 00:00:00+00:00',
  'lati

In [100]:
intersect_json['categories'][:10]

[{'id': 0,
  'name': 'Accipiter cooperii',
  'common_name': "Cooper's Hawk",
  'supercategory': 'Birds',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Aves',
  'order': 'Accipitriformes',
  'family': 'Accipitridae',
  'genus': 'Accipiter',
  'specific_epithet': 'cooperii',
  'image_dir_name': '03112_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_cooperii'},
 {'id': 1,
  'name': 'Accipiter gentilis',
  'common_name': 'Northern Goshawk',
  'supercategory': 'Birds',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Aves',
  'order': 'Accipitriformes',
  'family': 'Accipitridae',
  'genus': 'Accipiter',
  'specific_epithet': 'gentilis',
  'image_dir_name': '03113_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_gentilis'},
 {'id': 2,
  'name': 'Accipiter striatus',
  'common_name': 'Sharp-shinned Hawk',
  'supercategory': 'Birds',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Aves',
  'order': 'Accipitriformes',
  'family

In [101]:
intersect_json['annotations'][:10]

[{'id': 0, 'image_id': 0, 'category_id': 524},
 {'id': 1, 'image_id': 1, 'category_id': 607},
 {'id': 2, 'image_id': 2, 'category_id': 665},
 {'id': 3, 'image_id': 3, 'category_id': 388},
 {'id': 4, 'image_id': 4, 'category_id': 100},
 {'id': 5, 'image_id': 5, 'category_id': 84},
 {'id': 6, 'image_id': 6, 'category_id': 242},
 {'id': 7, 'image_id': 7, 'category_id': 138},
 {'id': 8, 'image_id': 8, 'category_id': 545},
 {'id': 9, 'image_id': 9, 'category_id': 205}]

In [102]:
intersect_json['licenses'] = train_annotations['licenses'].copy()

In [103]:
intersect_json['info'] = train_annotations['info'].copy()

In [105]:
intersect_json['info']['description'] = "zero-shot learning dataset combining iNaturalist 2021 with animal descriptions from animalia.com"
intersect_json['info']['contributor'] = "Matthew Jackson, Shresht Malik, Michael Matthews, Yousuf Mohamed-Ahmed"
intersect_json['info']['date_created'] = "2021-03-16"

In [108]:
# add description information
for cat in intersect_json['categories']:
    rows = df.loc[df['Species'] == cat['name'].lower()]
    cat['description'] = rows['Description'].values[0]

In [109]:
intersect_json['categories'][:10]

[{'id': 0,
  'name': 'Accipiter cooperii',
  'common_name': "Cooper's Hawk",
  'supercategory': 'Birds',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Aves',
  'order': 'Accipitriformes',
  'family': 'Accipitridae',
  'genus': 'Accipiter',
  'specific_epithet': 'cooperii',
  'image_dir_name': '03112_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_cooperii',
  'description': "Cooper's hawks are agile medium-sized raptors native to the North American continent. They have hooked bills that are well adapted for tearing the flesh of prey, as is typical of raptorial birds. As adults, they are a solid blue-gray color above and have a well-defined crown made of blackish-brown feathers above a paler nape and hindneck offset against their streaked rufous cheeks. Their tail is blue-gray on top and pale underneath, barred with three black bands. The adult’s underside shows a bit of whitish base color overlaid heavily with coarse, irregular rufous to cinnamon bands. Adu

In [110]:
with open('zanim.json', 'w') as outfile:
    json.dump(intersect_json, outfile)

In [52]:
train_annotations['images'][:2]

[{'id': 1,
  'width': 500,
  'height': 333,
  'file_name': 'train/04831_Animalia_Chordata_Mammalia_Rodentia_Sciuridae_Marmota_flaviventris/4653466f-f7b5-4e6c-985e-e059c1493b8f.jpg',
  'license': 0,
  'rights_holder': 'Michelle S. Koo',
  'date': '2010-07-06 22:17:00+00:00',
  'latitude': 38.86995,
  'longitude': -120.19326,
  'location_uncertainty': None},
 {'id': 2,
  'width': 500,
  'height': 375,
  'file_name': 'train/05015_Animalia_Chordata_Reptilia_Squamata_Iguanidae_Dipsosaurus_dorsalis/277c6a90-0432-4472-96b6-ed51a5cd21cf.jpg',
  'license': 1,
  'rights_holder': '105615097470186309865',
  'date': '2009-05-04 00:00:00+00:00',
  'latitude': 35.14218,
  'longitude': -116.10415,
  'location_uncertainty': None}]

In [24]:
train_annotations['categories'][0]

{'id': 0,
 'name': 'Lumbricus terrestris',
 'common_name': 'Common Earthworm',
 'supercategory': 'Animalia',
 'kingdom': 'Animalia',
 'phylum': 'Annelida',
 'class': 'Clitellata',
 'order': 'Haplotaxida',
 'family': 'Lumbricidae',
 'genus': 'Lumbricus',
 'specific_epithet': 'terrestris',
 'image_dir_name': '00000_Animalia_Annelida_Clitellata_Haplotaxida_Lumbricidae_Lumbricus_terrestris'}

In [93]:
train_annotations['annotations'][:10]

[{'id': 0, 'image_id': 0, 'category_id': 2912},
 {'id': 1, 'image_id': 1, 'category_id': 4831},
 {'id': 2, 'image_id': 2, 'category_id': 5015},
 {'id': 3, 'image_id': 3, 'category_id': 5163},
 {'id': 4, 'image_id': 4, 'category_id': 4983},
 {'id': 5, 'image_id': 5, 'category_id': 4482},
 {'id': 6, 'image_id': 6, 'category_id': 3238},
 {'id': 7, 'image_id': 7, 'category_id': 5804},
 {'id': 8, 'image_id': 8, 'category_id': 980},
 {'id': 9, 'image_id': 9, 'category_id': 3956}]

In [6]:
print(df["Full taxonomy"].values)

['Animalia_Chordata_Aves_Opisthocomiformes_Opisthocomidae_Opisthocomus_Opisthocomus hoazin'
 'Animalia_Chordata_Vertebrata_Reptilia_Testudines_Cryptodira_Kinosternidae_Sternotherus_Sternotherus odoratus'
 'Animalia_Chordata_Vertebrata_Mammalia_Carnivora_Caniformia_Pinnipedia_Phocidae_Phoca_Phoca vitulina'
 'Animalia_Chordata_Vertebrata_Mammalia_Primates_Haplorhini_Simiiformes_Cercopithecidae_Cercopithecinae_Cercopithecini_Cercopithecus_Cercopithecus mitis'
 'Animalia_Chordata_Vertebrata_Reptilia_Squamata_Serpentes_Pythonidae_Morelia_Morelia spilota'
 'Animalia_Chordata_Vertebrata_Reptilia_Testudines_Cryptodira_Testudinoidea_Testudinidae_Testudo_Testudo graeca'
 'Animalia_Chordata_Aves_Passeriformes_Passerellidae_Passerculus_Passerculus sandwichensis'
 'Animalia_Chordata_Aves_Neornithes_Neognathae_Neoaves_Pelecaniformes_Ardeidae_Egretta_Egretta thula'
 'Animalia_Chordata_Aves_Passeriformes_Corvidae_Perisoreus_Perisoreus canadensis'
 'Animalia_Chordata_Aves_Sphenisciformes_Spheniscidae_S

In [114]:
import os
dirs = []
path = "/Volumes/My Passport - Extra files/download-attempt-2/train/"
for x in tqdm(os.walk(path)):
    dirs.append(x[0])

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [117]:
with open("directories.txt", 'w') as output:
    for c in intersect_json['categories']:
        output.write(c['image_dir_name']+'\n')

In [2]:
f = open('zanim.json',) 
intersect_json = json.load(f) 

In [12]:
import subprocess
import os
path = "/Volumes/My Passport - Extra files/download-attempt-2/train/"
for c in tqdm(intersect_json['categories']):
    file = '"' + path + c['image_dir_name'] + '"'
    file = file.replace(" ","\\\\ ")
#     print(c['image_dir_name'])
# filefile
    cmd = f'"cp -R {file} /Users/yousuf/Desktop/zanim"'
    print(cmd)
#     os.system(cmd)
#     print(" ".join(["cp", "-R", file, '~/Desktop/zanim/']))
    subprocess.Popen(["cp", "-R", file, '/Users/yousuf/Desktop/zanim/'])
#     subprocess.Popen(["ls",'/Users/yousuf/Desktop/zanim/'])
#     subprocess.run(["cp", "-R", file, '~/Desktop/zanim/'])

  0%|          | 0/673 [00:00<?, ?it/s]

"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03112_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_cooperii" /Users/yousuf/Desktop/zanim"
"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03113_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_gentilis" /Users/yousuf/Desktop/zanim"
"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03115_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Accipiter_striatus" /Users/yousuf/Desktop/zanim"
"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03118_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Aquila_audax" /Users/yousuf/Desktop/zanim"
"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03119_Animalia_Chordata_Aves_Accipitriformes_Accipitridae_Aquila_chrysaetos" /Users/yousuf/Desktop/zanim"
"cp -R "/Volumes/My\\ Passport\\ -\\ Extra\\ files/download-attempt-2/train/03126_Animalia_C